In [1]:
import psycopg2

### Sample postgres table:

```
CREATE TABLE edge_table (
    id BIGSERIAL,
    osmid BIGINT,
    source BIGINT,
    target BIGINT,
    length FLOAT,
    name VARCHAR,
    highway VARCHAR,
    x1 FLOAT,
    y1 FLOAT,
    x2 FLOAT,
    y2 FLOAT);
```

### Postgres connection

In [2]:
conn = psycopg2.connect(
    host="localhost",
    database="krakow_150",
    user="postgres",
    password="postgres")

### Data loading

In [3]:
conn.autocommit = True
cursor = conn.cursor() 

In [4]:
drop_table = "DROP TABLE IF EXISTS edge_table CASCADE;"

create_table = """
    CREATE TABLE edge_table (
    id BIGSERIAL,
    osmid BIGINT,
    source BIGINT,
    target BIGINT,
    length FLOAT,
    name VARCHAR,
    highway VARCHAR,
    x1 FLOAT,
    y1 FLOAT,
    x2 FLOAT,
    y2 FLOAT,
    source_distance INT,
    target_distance INT
    );
"""

create_extension = """
    CREATE EXTENSION IF NOT EXISTS pgrouting CASCADE;
    CREATE EXTENSION IF NOT EXISTS postgis;
"""

create_index = """
    CREATE INDEX source_idx ON edge_table (source);
    CREATE INDEX target_idx ON edge_table (target);
    CREATE INDEX source_distance_idx ON edge_table (source_distance);
    CREATE INDEX target_distance_idx ON edge_table (target_distance);
"""

### Drop table (if exists)

In [5]:
cursor.execute(drop_table)

### Create extensions (pgRouting and postgis)

In [6]:
cursor.execute(create_extension)

### Create table

In [7]:
cursor.execute(create_table)

### Load data from csv

In [8]:
# TODO: remember to change path to .csv file
csv_file_name = "d:/AGH/NOVA/dm/data/krakow_150/edge_table_sql.csv"
copy_csv_query = "COPY edge_table(source, target, osmid, length, name, highway, x1, y1, x2, y2, source_distance, target_distance) FROM STDIN DELIMITER ',' CSV HEADER"

size_query = "SELECT pg_size_pretty( pg_database_size('krakow_distance') ); "


In [9]:
cursor.copy_expert(copy_csv_query, open(csv_file_name, "r", encoding="utf-8"))

### Create indexes on source and target

In [10]:
cursor.execute(create_index)

### Search queries

In [45]:
ids = (356926768, 2104495834) # short (zakamycze, betel)
# ids = (13822575, 11286442329) # random small

#long distance
ids = (4538716538, 432648629)

i = 0

In [46]:
dijkstra = f"""
    SELECT * FROM pgr_dijkstra(
    'SELECT id, source, target, length as cost, length as reverse_cost FROM edge_table',
    {ids[0]}, {ids[1]},
    directed => false);
"""

dijkstra_benchmark = f"""
    EXPLAIN ANALYZE 
    SELECT * FROM pgr_dijkstra(
    'SELECT id, source, target, length as cost, length as reverse_cost FROM edge_table',
    {ids[0]}, {ids[1]},
    directed => false);
"""

astar = f"""
    SELECT * FROM pgr_aStar(
    'SELECT id, source, target, length as cost, length as reverse_cost, x1, y1, x2, y2 FROM edge_table',
    {ids[0]}, {ids[1]},
    directed => false);
"""

astar_benchmark = f"""
    EXPLAIN ANALYZE
    SELECT * FROM pgr_aStar(
    'SELECT id, source, target, length as cost, length as reverse_cost, x1, y1, x2, y2 FROM edge_table',
    {ids[0]}, {ids[1]},
    directed => false);
"""


In [14]:
print(astar)
print(dijkstra)


    SELECT * FROM pgr_aStar(
    'SELECT id, source, target, length as cost, length as reverse_cost, x1, y1, x2, y2 FROM edge_table',
    356926768, 2104495834,
    directed => false);


    SELECT * FROM pgr_dijkstra(
    'SELECT id, source, target, length as cost, length as reverse_cost FROM edge_table',
    356926768, 2104495834,
    directed => false);



In [41]:
cursor.execute(astar)

result = cursor.fetchall()
columns = [desc[0] for desc in cursor.description]

print(columns)

for row in result:
    print(row)

InternalError_: BŁĄD:  invalid memory alloc request size 1080000000
CONTEXT:  funkcja SQL "pgr_astar" wyrażenie 1


In [32]:
cursor.execute(astar_benchmark)

result = cursor.fetchall()
columns = [desc[0] for desc in cursor.description]

print(columns)

for row in result:
    print(row)

['QUERY PLAN']
('Function Scan on pgr_astar  (cost=0.25..10.25 rows=1000 width=40) (actual time=1421.279..1421.289 rows=298 loops=1)',)
('Planning Time: 0.074 ms',)
('Execution Time: 1421.332 ms',)


In [26]:
ids = (262220894, 6949642867)
i=0 #0,1,2,3,4

astar_distance = f"""
    SELECT * FROM pgr_aStar(
    'SELECT id, source, target, length as cost, length as reverse_cost, x1, y1, x2, y2 
    FROM edge_table WHERE source_distance <= {i} AND target_distance <= {i}',
    {ids[0]}, {ids[1]},
    directed => false);
"""

#astar crushes between 2 and 3

cursor.execute(astar_distance)

result = cursor.fetchall()
columns = [desc[0] for desc in cursor.description]

print(columns)

for row in result:
    print(row)

['seq', 'path_seq', 'node', 'edge', 'cost', 'agg_cost']
(1, 1, 356926768, 15258509, 22.495, 0.0)
(2, 2, 2583143012, 15573664, 30.134, 22.495)
(3, 3, 2583143013, 15334842, 16.071, 52.629000000000005)
(4, 4, 1364114321, 15334843, 7.768, 68.7)
(5, 5, 2000518505, 15258510, 13.455, 76.468)
(6, 6, 356926771, 15258511, 31.201, 89.923)
(7, 7, 356926775, 15258513, 21.388, 121.124)
(8, 8, 2583143014, 15573668, 15.735, 142.512)
(9, 9, 2583143015, 15394563, 27.355, 158.247)
(10, 10, 1752636499, 15394562, 14.562, 185.602)
(11, 11, 2583137150, 15573588, 12.532, 200.16400000000002)
(12, 12, 2583137151, 15573590, 13.823, 212.69600000000003)
(13, 13, 2583137152, 15573592, 14.322, 226.51900000000003)
(14, 14, 2583137153, 15573594, 15.154, 240.84100000000004)
(15, 15, 2583137154, 15573596, 9.142, 255.99500000000003)
(16, 16, 2583137155, 15573598, 22.462, 265.13700000000006)
(17, 17, 2583137156, 15573600, 11.333, 287.59900000000005)
(18, 18, 2583137157, 15573602, 11.269, 298.9320000000001)
(19, 19, 258313

In [47]:
cursor.execute(dijkstra)

result = cursor.fetchall()
columns = [desc[0] for desc in cursor.description]

print(columns)

for row in result:
    print(row)

['seq', 'path_seq', 'start_vid', 'end_vid', 'node', 'edge', 'cost', 'agg_cost']
(1, 1, 4538716538, 432648629, 4538716538, 6310307, 51.305, 0.0)
(2, 2, 4538716538, 432648629, 4538716573, 6332783, 73.127, 51.305)
(3, 3, 4538716538, 432648629, 6473506347, 6279928, 6.658, 124.43199999999999)
(4, 4, 4538716538, 432648629, 3225643862, 6279924, 18.208, 131.08999999999997)
(5, 5, 4538716538, 432648629, 3225643657, 6279925, 19.848, 149.29799999999997)
(6, 6, 4538716538, 432648629, 5928783367, 6279912, 4.901, 169.14599999999996)
(7, 7, 4538716538, 432648629, 3225643646, 6279911, 3.754, 174.04699999999997)
(8, 8, 4538716538, 432648629, 3225643645, 6279910, 4.629, 177.80099999999996)
(9, 9, 4538716538, 432648629, 3225643639, 6279906, 12.283, 182.42999999999995)
(10, 10, 4538716538, 432648629, 2192904813, 6243673, 20.873, 194.71299999999994)
(11, 11, 4538716538, 432648629, 2270933916, 6248124, 42.203, 215.58599999999993)
(12, 12, 4538716538, 432648629, 2344914474, 6232930, 5.798, 257.78899999999993

In [33]:
cursor.execute(dijkstra_benchmark)

result = cursor.fetchall()
columns = [desc[0] for desc in cursor.description]

print(columns)

for row in result:
    print(row)

['QUERY PLAN']
('Function Scan on pgr_dijkstra  (cost=0.25..10.25 rows=1000 width=56) (actual time=1110.136..1110.147 rows=298 loops=1)',)
('Planning Time: 0.043 ms',)
('Execution Time: 1110.174 ms',)


### Disconnect

In [30]:
conn.commit() 
conn.close() 